# 1. Elasticsearch 서버 실행

In [1]:
import os
import json
from elasticsearch import Elasticsearch, helpers
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(['elasticsearch-8.8.0/bin/elasticsearch'],
                stdout=PIPE, stderr=STDOUT,
)

# 인스턴스를 로드하는 데 약간의 시간이 걸림
import time
time.sleep(60)

# 2. Elasticsearch client 생성

In [2]:
import os
import json
from elasticsearch import Elasticsearch, helpers

es_username = 'elastic'

# PASSWORD 입력
es_password = '=+jar*Nd3lPkY+Z5qbD_'

# Elasticsearch client 생성
es = Elasticsearch(
    ['https://localhost:9200'],
    basic_auth=(es_username, es_password),
    ca_certs="elasticsearch-8.8.0/config/certs/http_ca.crt")

# Elasticsearch client 정보 확인
print(es.info())

{'name': 'iseonhos-MacBook-Air.local', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'tR1YDniKTMWaTLEmCMliYQ', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


# 3. Elasticsearch 색인 생성

In [3]:
settings = {
    "analysis": {
        "analyzer" : {
            "nori": {
                "type" : "custom",
                "tokenizer" : "nori_tokenizer",
                "decompound_mode" : "mixed",
                "filter" : ["nori_posfilter"]
            }
        },
        "filter": {
            "nori_posfilter": {
                "type": "nori_part_of_speech",
                "stoptags" : ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]
            }
        }
    }
}

mappings = {
    "properties" : {
        "content": {"type": "text", "analyzer": "nori"},
        "embeddings" : {
            "type": "dense_vector",
            "dims": 768,
            "index": True,
            "similarity": "l2_norm"
        }
    }
}

In [4]:
# 색인 생성
def create_es_index(index, settings, mappings):
    if es.indices.exists(index = index):
        es.indices.delete(index = index)
    
    es.indices.create(index = index, settings = settings, mappings = mappings)


create_es_index("test", settings, mappings)

# 4. 데이터 임베딩

In [5]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

def get_embedding(sentences):
    return model.encode(sentences)

def get_embeddings_in_batches(docs, batch_size = 100):
    batch_embeddings = []
    for i in tqdm(range(0, len(docs), batch_size)):
        batch = docs[i:i+batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = get_embedding(contents)
        batch_embeddings.extend(embeddings)

    return batch_embeddings

/opt/anaconda3/envs/rag/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# 데이터 가져요기
with open("data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]

# docs 임베딩
embeddings = get_embeddings_in_batches(docs)

100%|██████████| 43/43 [03:08<00:00,  4.38s/it]


# 5. 임베딩 데이터 인덱스에 저장하기

In [7]:
from elasticsearch import helpers

index_docs = []
for doc, embedding in zip(docs, embeddings):
    doc["embeddings"] = embedding.tolist()
    index_docs.append(doc)

def bulk_add(index, docs):
    actions = [
        {
            '_index' : index,
            '_source' : doc
        }
        for doc in docs
    ]

    return helpers.bulk(es, actions)

# 임베딩된 docs를 index에 bulk 단위로 추가
ret = bulk_add("test", index_docs)

# 6. 테스트 해보기

In [8]:
test_query = "금성이 다른 행성보다 밝게 보이는 이유는 무엇인가요?"

In [9]:
# 놀라운 사실 이것은 KNN이 아니라 HNSW 알고리즘을 사용하는 것이었습니다.
# knn이라고 주는것은 그저 elasticsearch에서 이름을 그렇게 사용하는 것이 었어요... 소오오오오오름

def dense_retrieve(query_str, size):
    query_embedding = get_embedding([query_str])[0]

    knn = {
        "field" : "embeddings",
        "query_vector" : query_embedding.tolist(),
        "k" : size,
        "num_candidates" : 100
    }

    return es.search(index="test", knn=knn)

def sparse_retrieve(query_str, size):
    query = {
        "match" : {
            "content" : {
                "query" : query_str
            }
        }
    }

    return es.search(index="test", query = query, size = size, sort="_score")

# 역색인 검색 결과(점수 : BM25)
search_result_retrieve = sparse_retrieve(test_query, 3)

for rst in search_result_retrieve['hits']['hits']:
    print(rst)
    print("score: ", rst['_score'], "source: ", rst['_source']['content'])

# KNN 검색 결과
search_result_retrieve = dense_retrieve(test_query, 3)

for rst in search_result_retrieve['hits']['hits']:
    print("socre: ", rst['_score'], "source: ", rst['_source']['content'])

{'_index': 'test', '_id': 'du5Ho5UBJTgUTksmMvZW', '_score': 32.46292, '_source': {'docid': '464ace62-ddf2-423d-a5d7-2f17e6785c8e', 'src': 'ko_ai2_arc__ARC_Challenge__train', 'content': '금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.', 'embeddings': [-0.21888276934623718, -0.39615678787231445, -0.4124566912651062, -0.2202705293893814, 0.26675331592559814, -0.677114725112915, 1.0647697448730469, -0.8683406114578247, 0.3121263086795807, -0.2870643436908722, -0.30825501680374146, -0.18074484169483185, 0.03387121111154556, 0.00035895142355002463, -0.2293367236852646, -0.17151030898094177, 0.11902165412902832, -0.5617472529411316, 0.4231579601764679, 0.4816335439682007, 0.35183534026145935, 0.4834263026714325, -0.4751304090023041, 0.6908711194992065, 1.0748343467712402, -0.6240761876